In [ ]:
globals().clear
import time
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
from datetime import datetime

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
import xgboost as xgb

In [ ]:
# Load Dataset
df = pd.read_csv('df.csv')

In [ ]:
# list of tickers for stocks in our data set. Sort the ticker list since our df will be alphabetically arranged.
tickers = ["AAPL", 'XOM', 'IBM', 'KO', 'CVX', 'BA', 'PFE', 'MSFT', 'T', 'WMT',
       'F', 'NFLX', 'JPM', 'MCD', 'GE', 'NVDA', 'JNJ', 'BAC', 'C', 'AMZN',
       'INTC', 'CSCO', 'TSLA', 'GOOGL', 'AMD', 'BABA', 'VZ', 'DIS',
       'META', 'PLTR']
tickers.sort()

In [ ]:
# For this specific trial drop PLTR since data is not complete
df = df[df.stock_ID != "PLTR"]

In [ ]:
df['DATETIME']= pd.to_datetime(df['DATETIME'], format='%m/%d/%Y %H:%M')

In [ ]:
# Sort by time so that the first 29 rows occupy the first time value for all the stocks.
# Fill in 0's for missing values for now.
df = df.set_index('DATETIME')
df = df.fillna(0)

In [ ]:
# Select the length of the df ; For this file we need 16 months
# We use the first 12 months to make the first prediction, then shift window
# Then we will repeat this for the next 4 months (Hence total is 16 months needed)
df = df.sort_index().loc['2021-09-01':'2022-12-30']

In [ ]:
#Set the DATETIME for fecha
DATETIME = df.index.values

In [ ]:
# Sort dataframe such that it is both in sequential order, and also in alphabetical order for each day
#(i.e first entry for each time entry should be AAPL, and last should be XOM).
df.sort_values(["DATETIME", "stock_ID"], inplace=True)

In [ ]:
file_name = 'XGB.60d.1d'
num_companies = 29  # 29 companies in our dataset.
step_rows  = 24 * num_companies  # 24 time periods per day per stock
total_rows = len(df['2021-09-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(df['2021-09-01':'2022-08-31'])  # Define length of training window

scale_X = MinMaxScaler()
model = XGBRegressor(n_estimators=300, max_depth=100, learning_rate=0.1, objective='reg:squarederror'
                     , alpha=10, tree_method="gpu_hist")

result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = df.iloc[i:train_rows + i, 1:]
    test   = df.iloc[train_rows + i:train_rows + i + step_rows, 1:]
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]


    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model.fit(X_train, y_train)
    y_hat = model.predict(X_test)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    result = pd.concat([result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

Count Down: 81
Count Down: 80
Count Down: 79
Count Down: 78
Count Down: 77
Count Down: 76
Count Down: 75
Count Down: 74
Count Down: 73
Count Down: 72
Count Down: 71
Count Down: 70
Count Down: 69
Count Down: 68
Count Down: 67
Count Down: 66
Count Down: 65
Count Down: 64
Count Down: 63
Count Down: 62
Count Down: 61
Count Down: 60
Count Down: 59
Count Down: 58
Count Down: 57
Count Down: 56
Count Down: 55
Count Down: 54
Count Down: 53
Count Down: 52
Count Down: 51
Count Down: 50
Count Down: 49
Count Down: 48
Count Down: 47
Count Down: 46
Count Down: 45
Count Down: 44
Count Down: 43
Count Down: 42
Count Down: 41
Count Down: 40
Count Down: 39
Count Down: 38
Count Down: 37
Count Down: 36
Count Down: 35
Count Down: 34
Count Down: 33
Count Down: 32
Count Down: 31
Count Down: 30
Count Down: 29
Count Down: 28
Count Down: 27
Count Down: 26
Count Down: 25
Count Down: 24
Count Down: 23
Count Down: 22
Count Down: 21
Count Down: 20
Count Down: 19
Count Down: 18
Count Down: 17
Count Down: 16
Count Down

In [ ]:
# Create Dataframe for the results
metrics_df = pd.DataFrame(columns = ['stock_ID', "RMSE", "MAPE", "MPE", "MTT"])

In [ ]:
result[25::29]

,DATETIME,ACTUAL,PREDICTED,DIFFERENCE,TRAIN_DURATION
25,2022-09-01 09:30:00,274.66,260.089264,14.570736,43.416939
54,2022-09-01 09:45:00,273.62,258.003113,15.616887,43.416939
83,2022-09-01 10:00:00,271.36,260.223389,11.136611,43.416939
112,2022-09-01 10:15:00,273.14,258.052521,15.087479,43.416939
141,2022-09-01 10:30:00,269.54,257.443726,12.096274,43.416939
...,...,...,...,...,...
56256,2022-12-30 14:15:00,121.46,115.311661,6.148339,104.625571
56285,2022-12-30 14:30:00,121.3,115.359711,5.940289,104.625571
56314,2022-12-30 14:45:00,121.51,118.275902,3.234098,104.625571
56343,2022-12-30 15:00:00,122.42,121.353554,1.066446,104.625571


In [ ]:
def mean_positive_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    error = np.mean(np.maximum((y_pred - y_true),0))
    return error

In [ ]:
new_rows = []
for i in range(len(tickers)):
    stock_result = result[i::29]
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
    # Calculate MAPE %
    mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
    # Calculate MPE %
    mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
    # Calculate MTT in seconds
    mtt = np.mean(stock_result['TRAIN_DURATION'])
    new_row = pd.Series([tickers[i],rmse, mape, mpe, mtt], index=metrics_df.columns)
    new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)

In [ ]:
metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT
0,AAPL,2.897865,1.477039,1.388042,105.057378
1,AMD,2.405507,2.731123,1.440116,105.057378
2,AMZN,3.554116,2.288400,1.578920,105.057378
3,BA,5.667668,2.506331,1.039656,105.057378
4,BABA,4.195752,3.788328,1.345082,105.057378
5,BAC,0.964061,2.129059,0.364014,105.057378
6,C,0.938909,1.584965,0.436738,105.057378
7,CSCO,0.745434,1.180004,0.155210,105.057378
8,CVX,3.872372,1.784248,1.139435,105.057378
9,DIS,3.524677,2.230591,1.183549,105.057378


In [ ]:
avg_values=metrics_df.mean()
avg_df = pd.DataFrame(avg_values).T
avg_df